This script is aiming to find the proper script to make a classicification of the images.

# Import pacakage

In [1]:
import tensorflow as tf
import numpy as np

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import PIL
#import requests
from io import BytesIO
from IPython.display import display
%matplotlib inline

import os
import scipy

# Use in the model
from tensorflow.keras.layers import Dense, Dropout

# Load Images

In [2]:
# Get the name of all the labels
home_path = '/home/bowen/Documents/Ecomed/'
data_path = '/home/bowen/Documents/Ecomed/dataset/dataset_big_crop'
# class_names = os.listdir(data_path)
class_names = ["background", "pharmaceutical", "sharps", "trace_chemo"]

In [3]:
# Set up data generators that can read images from our dataset into Keras.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.5)

train_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='binary',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='binary',
        subset='validation')

Found 16115 images belonging to 4 classes.
Found 16115 images belonging to 4 classes.


In [7]:
# # Show several images
# data, labels = validation_generator.next()

# for i, sample in enumerate(data):
#     plt.imshow(sample)
#     print(class_names[int(labels[i])])
#     plt.show()

# MobileNet Model

In [8]:
gpu_index = 3

In [9]:
# Import mobilenet
# with tf.device('/gpu:' + str(gpu_index)):
mobilenet = tf.keras.applications.MobileNet(include_top=False, input_shape=[224, 224, 3], weights='imagenet')

In [10]:
# Add new output layers.
# with tf.device('/gpu:' + str(gpu_index)):
x = mobilenet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = tf.keras.layers.Dense(4, activation='softmax')(x)

# Combine feature layers with our new output layers.
model = tf.keras.Model(inputs=mobilenet.input, outputs=preds)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

# Train the Model

In [11]:
# defintion of optimizer
# with tf.device('/gpu:' + str(gpu_index)):
# opt = tf.keras.optimizers.SGD(0.001)
opt = tf.train.GradientDescentOptimizer(0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
# definition of checkpoint
# with tf.device('/gpu:' + str(gpu_index)):
checkpoint_path = os.path.join(home_path, 'models', 'model_big4', "model_big4.h5")
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, save_weights_only=True)

In [6]:
#model = tf.keras.models.load_model(os.path.join(home_path + 'models', "model_new_layer.h5"))

In [13]:
# train model
with tf.device('/gpu:3'):
    model.fit_generator(generator=train_generator,
                        validation_data=validation_generator, 
                        epochs=200,
                        callbacks=[cp_callback])

Epoch 1/200
672/672 [==============================] - 418s 622ms/step - loss: 1.0857 - acc: 0.5354 - val_loss: 0.5925 - val_acc: 0.8086
Epoch 2/200
672/672 [==============================] - 411s 611ms/step - loss: 0.4972 - acc: 0.8129 - val_loss: 0.3880 - val_acc: 0.8824
Epoch 3/200
672/672 [==============================] - 412s 614ms/step - loss: 0.3203 - acc: 0.8857 - val_loss: 0.2875 - val_acc: 0.8952
Epoch 4/200
672/672 [==============================] - 412s 613ms/step - loss: 0.2257 - acc: 0.9216 - val_loss: 0.2521 - val_acc: 0.9176
Epoch 5/200
672/672 [==============================] - 412s 613ms/step - loss: 0.1779 - acc: 0.9397 - val_loss: 0.2444 - val_acc: 0.9257
Epoch 6/200
672/672 [==============================] - 411s 611ms/step - loss: 0.1557 - acc: 0.9463 - val_loss: 0.2872 - val_acc: 0.9225
Epoch 7/200
672/672 [==============================] - 415s 618ms/step - loss: 0.1317 - acc: 0.9546 - val_loss: 0.2561 - val_acc: 0.9258
Epoch 8/200
672/672 [====================

672/672 [==============================] - 413s 615ms/step - loss: 0.0104 - acc: 0.9967 - val_loss: 0.3958 - val_acc: 0.9244
Epoch 61/200
672/672 [==============================] - 411s 611ms/step - loss: 0.0088 - acc: 0.9970 - val_loss: 0.3826 - val_acc: 0.9292
Epoch 62/200
672/672 [==============================] - 412s 613ms/step - loss: 0.0075 - acc: 0.9980 - val_loss: 0.3661 - val_acc: 0.9307
Epoch 63/200
672/672 [==============================] - 411s 611ms/step - loss: 0.0059 - acc: 0.9983 - val_loss: 0.3596 - val_acc: 0.9329
Epoch 64/200
672/672 [==============================] - 413s 615ms/step - loss: 0.0065 - acc: 0.9981 - val_loss: 0.3753 - val_acc: 0.9335
Epoch 65/200
672/672 [==============================] - 412s 613ms/step - loss: 0.0064 - acc: 0.9982 - val_loss: 0.4290 - val_acc: 0.9225
Epoch 66/200
672/672 [==============================] - 413s 615ms/step - loss: 0.0078 - acc: 0.9975 - val_loss: 0.3739 - val_acc: 0.9343
Epoch 67/200
672/672 [=========================

KeyboardInterrupt: 

In [14]:
tf.contrib.saved_model.save_keras_model(model, home_path + 'models/model_big4')

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tens

b'/home/bowen/Documents/Ecomed/models/model_big4/1552404675'